In [2]:
import tensorflow as tf
tf.random.set_seed(1)
rnn_layer = tf.keras.layers.SimpleRNN( units=2, use_bias=True, return_sequences=True)
rnn_layer.build(input_shape=(None, None, 5))
w_xh, w_oo, b_h = rnn_layer.weights
print('W_xh shape:', w_xh.shape)
print('W_oo shape:', w_oo.shape)
print('b_h shape:', b_h.shape)

W_xh shape: (5, 2)
W_oo shape: (2, 2)
b_h shape: (2,)


In [3]:
x_seq = tf.convert_to_tensor(
    [[1.0]*5, [2.0]*5, [3.0]*5],
    dtype=tf.float32)
## output of SimepleRNN:
output = rnn_layer(tf.reshape(x_seq, shape=(1, 3, 5)))
## manually computing the output:
out_man = []
for t in range(len(x_seq)):
    xt = tf.reshape(x_seq[t], (1, 5))
    print('Time step {} =>'.format(t))
    print(' Input :', xt.numpy())
 
    ht = tf.matmul(xt, w_xh) + b_h 
    print(' Hidden :', ht.numpy())
 
    if t>0:
        prev_o = out_man[t-1]
    else:
        prev_o = tf.zeros(shape=(ht.shape))
 
    ot = ht + tf.matmul(prev_o, w_oo)
    ot = tf.math.tanh(ot)
    out_man.append(ot)
    print(' Output (manual) :', ot.numpy())
    print(' SimpleRNN output:'.format(t), output[0][t].numpy())
    print()

Time step 0 =>
 Input : [[1. 1. 1. 1. 1.]]
 Hidden : [[-2.7312071   0.04119337]]
 Output (manual) : [[-0.9915492   0.04117008]]
 SimpleRNN output: [-0.9915492   0.04117008]

Time step 1 =>
 Input : [[2. 2. 2. 2. 2.]]
 Hidden : [[-5.4624143   0.08238673]]
 Output (manual) : [[-0.9999512  0.786872 ]]
 SimpleRNN output: [-0.9999512  0.786872 ]

Time step 2 =>
 Input : [[3. 3. 3. 3. 3.]]
 Hidden : [[-8.193621    0.12358022]]
 Output (manual) : [[-0.9999993  0.7737364]]
 SimpleRNN output: [-0.9999993  0.7737364]



In [4]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd

In [5]:
import os
import gzip
import shutil
with gzip.open('movie_data.csv.gz', 'rb') as f_in, open('movie_data.csv', 'wb') as f_out:
    shutil.copyfileobj(f_in, f_out)

In [6]:
df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.tail()

,review,sentiment
49995,"OK, lets start with the best. the building. al...",0
49996,The British 'heritage film' industry is out of...,0
49997,I don't even know where to begin on this one. ...,0
49998,Richard Tyler is a little boy who is scared of...,0
49999,I waited long to watch this movie. Also becaus...,1


In [7]:
target = df.pop('sentiment')
ds_raw = tf.data.Dataset.from_tensor_slices(
    (df.values, target.values))
## inspection:
for ex in ds_raw.take(3):
    tf.print(ex[0].numpy()[0][:50], ex[1])

b'In 1974, the teenager Martha Moxley (Maggie Grace)' 1
b'OK... so... I really like Kris Kristofferson and h' 0
b'***SPOILER*** Do not read this, if you think about' 0


In [8]:
# 1. Buat objek dataser TensorFlow dan pisahkan menjadi partisi training, testing, dan validasi set yang terpisah.
tf.random.set_seed(1)
ds_raw = ds_raw.shuffle(
 50000, reshuffle_each_iteration=False)
ds_raw_test = ds_raw.take(25000)
ds_raw_train_valid = ds_raw.skip(25000)
ds_raw_train = ds_raw_train_valid.take(20000)
ds_raw_valid = ds_raw_train_valid.skip(20000)

In [9]:
from collections import Counter
try:
    tokenizer = tfds.features.text.Tokenizer()
except AttributeError:
    tokenizer = tfds.deprecated.text.Tokenizer()
token_counts = Counter()
for example in ds_raw_train:
    tokens = tokenizer.tokenize(example[0].numpy()[0])
    token_counts.update(tokens)
print('Vocab-size:', len(token_counts))

Vocab-size: 87007


In [10]:

## Step 3: encoding each unique token into integers
try:
    encoder = tfds.features.text.TokenTextEncoder(token_counts)
except AttributeError:
    encoder = tfds.deprecated.text.TokenTextEncoder(token_counts)
example_str = 'This is an example!'
encoder.encode(example_str)


[232, 9, 270, 1123]

In [11]:

## Step 3-A: define the function for transformation
def encode(text_tensor, label):
    text = text_tensor.numpy()[0]
    encoded_text = encoder.encode(text)
    return encoded_text, label
## Step 3-B: wrap the encode function to a TF Op.
def encode_map_fn(text, label):
    return tf.py_function(encode, inp=[text, label],
                        Tout=(tf.int64, tf.int64))


In [12]:
ds_train = ds_raw_train.map(encode_map_fn)
ds_valid = ds_raw_valid.map(encode_map_fn)
ds_test = ds_raw_test.map(encode_map_fn)
tf.random.set_seed(1)
for example in ds_train.shuffle(1000).take(5):
    print('Sequence length:', example[0].shape)
example


Sequence length: (24,)
Sequence length: (179,)
Sequence length: (262,)
Sequence length: (535,)
Sequence length: (130,)


(<tf.Tensor: shape=(130,), dtype=int64, numpy=
 array([  579,  1296,    32,   425,    40,   763,  9267,    65,   280,
          308,     6,   481,   155,   473,     2,     3,   684,     9,
          781,   176,   959,   730,  3917,    67,  9905,    13,   277,
           24,    35,   371, 16368,     6,    14, 17231,    29,   187,
         1651,   489,   503,   480,   143,    32,   270,  5851,  2402,
           13,  3592,  3443,   425,  3313,   256,   257,  1577,   117,
            8,   698,   270,   564,    56,     8,    42,  7517,  2629,
          820,    25,    60,    79,   343,    32,   645,    14,   528,
          241,    32,  1980,     8,    56,     8,    42,  1364,   573,
         5183,    43,    12,  3870,    32,   312,   642,   251,  1401,
        17232,     8,   698,   257,   750,     2,     9,    76,   235,
            8,    42,   235,   840,   666,   258, 17233,   419,    32,
        17234,   585,   420,   840,    25,    40,    13,    14,   198,
          266,   623,   173,  

In [13]:

## Take a small subset
ds_subset = ds_train.take(8)
for example in ds_subset:
    print('Individual size:', example[0].shape)
## batching the datasets
ds_batched = ds_subset.padded_batch(
    4, padded_shapes=([-1], []))
for batch in ds_batched:
    print('Batch dimension:', batch[0].shape)


Individual size: (119,)
Individual size: (688,)
Individual size: (308,)
Individual size: (204,)
Individual size: (326,)
Individual size: (240,)
Individual size: (127,)
Individual size: (453,)
Batch dimension: (4, 688)
Batch dimension: (4, 453)


In [14]:

## batching the datasets
train_data = ds_train.padded_batch(
    32, padded_shapes=([-1],[]))
valid_data = ds_valid.padded_batch(
    32, padded_shapes=([-1],[]))
test_data = ds_test.padded_batch(
    32, padded_shapes=([-1],[]))


In [15]:

from tensorflow.keras.layers import Embedding
model = tf.keras.Sequential()
model.add(Embedding(input_dim=100,
                    output_dim=6,
                    input_length=20,
                    name='embed-layer'))
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed-layer (Embedding)     (None, 20, 6)             600       
                                                                 
Total params: 600
Trainable params: 600
Non-trainable params: 0
_________________________________________________________________


In [16]:

## An example of building a RNN model
## with SimpleRNN layer
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dense
model = Sequential()
model.add(Embedding(1000, 32))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32))
model.add(Dense(1))
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          32000     
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, None, 32)          2080      
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 32)                2080      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 36,193
Trainable params: 36,193
Non-trainable params: 0
_________________________________________________________________


In [17]:
from tensorflow.keras.layers import LSTM
model = Sequential()
model.add(Embedding(10000, 32))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(32))
model.add(Dense(1))
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 32)          320000    
                                                                 
 lstm (LSTM)                 (None, None, 32)          8320      
                                                                 
 lstm_1 (LSTM)               (None, 32)                8320      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 336,673
Trainable params: 336,673
Non-trainable params: 0
_________________________________________________________________


In [18]:
embedding_dim = 20
vocab_size = len(token_counts) + 2
tf.random.set_seed(1)
## build the model
bi_lstm_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        name='embed-layer'),
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64, name='lstm-layer'),
        name='bidir-lstm'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')])
bi_lstm_model.summary()
## compile and train:
bi_lstm_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=['accuracy'])
history = bi_lstm_model.fit(
    train_data,
    validation_data=valid_data,
    epochs=1)
## evaluate on the test data
test_results= bi_lstm_model.evaluate(test_data)
print('Test Acc.: {:.2f}%'.format(test_results[1]*100))

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed-layer (Embedding)     (None, None, 20)          1740180   
                                                                 
 bidir-lstm (Bidirectional)  (None, 128)               43520     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,792,021
Trainable params: 1,792,021
Non-trainable params: 0
_________________________________________________________________
782/782 [==============================] - 292s 372ms/step - loss: 0.3801 - accuracy: 0.8376
Test Acc.: 83.76%


In [19]:
def preprocess_datasets(
    ds_raw_train,
    ds_raw_valid,
    ds_raw_test,
    max_seq_length=None,
    batch_size=32):
 
 ## Step 1: (already done => creating a dataset)
 ## Step 2: find unique tokens
    try:
        tokenizer = tfds.features.text.Tokenizer()
    except AttributeError:
        tokenizer = tfds.deprecated.text.Tokenizer()
    token_counts = Counter()
    for example in ds_raw_train:
        tokens = tokenizer.tokenize(example[0].numpy()[0])
        if max_seq_length is not None:
            tokens = tokens[-max_seq_length:]
        token_counts.update(tokens)
    print('Vocab-size:', len(token_counts))
 ## Step 3: encoding the texts
    try:
        encoder = tfds.features.text.TokenTextEncoder(token_counts)
    except AttributeError:
        encoder = tfds.deprecated.text.TokenTextEncoder(token_counts)
    def encode(text_tensor, label):
        text = text_tensor.numpy()[0]
        encoded_text = encoder.encode(text)
        if max_seq_length is not None:
            encoded_text = encoded_text[-max_seq_length:]
        return encoded_text, label
    def encode_map_fn(text, label):
        return tf.py_function(encode, inp=[text, label],
                              Tout=(tf.int64, tf.int64))
    ds_train = ds_raw_train.map(encode_map_fn)
    ds_valid = ds_raw_valid.map(encode_map_fn)
    ds_test = ds_raw_test.map(encode_map_fn)
## Step 4: batching the datasets
    train_data = ds_train.padded_batch(
        batch_size, padded_shapes=([-1],[]))
    valid_data = ds_valid.padded_batch(
        batch_size, padded_shapes=([-1],[]))
    test_data = ds_test.padded_batch(
        batch_size, padded_shapes=([-1],[]))
    return (train_data, valid_data,
        test_data, len(token_counts))


In [20]:
def build_rnn_model(embedding_dim, vocab_size,
                    recurrent_type='SimpleRNN',
                    n_recurrent_units=64,
                    n_recurrent_layers=1,
                    bidirectional=True):
    tf.random.set_seed(1)
 # build the model
    model = tf.keras.Sequential()
 
    model.add(
        Embedding(
            input_dim=vocab_size,
            output_dim=embedding_dim,
            name='embed-layer')
    )
 
    for i in range(n_recurrent_layers):
        return_sequences = (i < n_recurrent_layers-1)
 
        if recurrent_type == 'SimpleRNN':
            recurrent_layer = SimpleRNN(
                units=n_recurrent_units,
                return_sequences=return_sequences,
                name='simprnn-layer-{}'.format(i))
        elif recurrent_type == 'LSTM':
            recurrent_layer = LSTM(
                units=n_recurrent_units,
                return_sequences=return_sequences,
                name='lstm-layer-{}'.format(i))
        elif recurrent_type == 'GRU':
            recurrent_layer = GRU(
                units=n_recurrent_units,
                return_sequences=return_sequences,
                name='gru-layer-{}'.format(i))
        if bidirectional:
            recurrent_layer = Bidirectional(
                recurrent_layer, name='bidir-'+recurrent_layer.name)
        model.add(recurrent_layer)
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    return model


In [21]:
from tensorflow.keras.layers import Bidirectional
batch_size = 32
embedding_dim = 20
max_seq_length = 100
train_data, valid_data, test_data, n = preprocess_datasets(
    ds_raw_train, ds_raw_valid, ds_raw_test,
    max_seq_length=max_seq_length,
    batch_size=batch_size
)
vocab_size = n + 2
rnn_model = build_rnn_model(
    embedding_dim, vocab_size,
    recurrent_type='SimpleRNN',
    n_recurrent_units=64,
    n_recurrent_layers=1,
    bidirectional=True)
rnn_model.summary()


Vocab-size: 58063
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed-layer (Embedding)     (None, None, 20)          1161300   
                                                                 
 bidir-simprnn-layer-0 (Bidi  (None, 128)              10880     
 rectional)                                                      
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,180,501
Trainable params: 1,180,501
Non-trainable params: 0
_________________________________________________________________


In [22]:
rnn_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                  metrics=['accuracy'])
history = rnn_model.fit(
    train_data,
    validation_data=valid_data,
    epochs=2)


Epoch 1/2
625/625 [==============================] - 79s 125ms/step - loss: 0.7026 - accuracy: 0.5034 - val_loss: 0.7207 - val_accuracy: 0.4844
Epoch 2/2
625/625 [==============================] - 78s 125ms/step - loss: 0.7040 - accuracy: 0.5019 - val_loss: 0.7133 - val_accuracy: 0.4866


In [23]:
results = rnn_model.evaluate(test_data)

782/782 [==============================] - 35s 45ms/step - loss: 0.7086 - accuracy: 0.5003


In [24]:
print('Test Acc.: {:.2f}%'.format(results[1]*100))


Test Acc.: 50.03%


In [25]:
lstm_model = build_rnn_model(
    embedding_dim, vocab_size,
    recurrent_type='LSTM',
    n_recurrent_units=64,
    n_recurrent_layers=1,
    bidirectional=True)
lstm_model.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed-layer (Embedding)     (None, None, 20)          1161300   
                                                                 
 bidir-lstm-layer-0 (Bidirec  (None, 128)              43520     
 tional)                                                         
                                                                 
 dense_6 (Dense)             (None, 64)                8256      
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,213,141
Trainable params: 1,213,141
Non-trainable params: 0
_________________________________________________________________


In [26]:
lstm_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                   loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                   metrics=['accuracy'])
history = lstm_model.fit(
    train_data,
    validation_data=valid_data,
    epochs=10)

Epoch 1/10
625/625 [==============================] - 140s 220ms/step - loss: 0.4555 - accuracy: 0.7742 - val_loss: 0.3455 - val_accuracy: 0.8502
Epoch 2/10
625/625 [==============================] - 135s 215ms/step - loss: 0.1990 - accuracy: 0.9245 - val_loss: 0.3786 - val_accuracy: 0.8500
Epoch 3/10
625/625 [==============================] - 134s 215ms/step - loss: 0.1025 - accuracy: 0.9663 - val_loss: 0.4908 - val_accuracy: 0.8396
Epoch 4/10
625/625 [==============================] - 132s 211ms/step - loss: 0.0698 - accuracy: 0.9763 - val_loss: 0.7160 - val_accuracy: 0.8094
Epoch 5/10
625/625 [==============================] - 131s 210ms/step - loss: 0.0455 - accuracy: 0.9861 - val_loss: 0.6782 - val_accuracy: 0.8380
Epoch 6/10
625/625 [==============================] - 132s 210ms/step - loss: 0.0254 - accuracy: 0.9925 - val_loss: 0.7743 - val_accuracy: 0.8364
Epoch 7/10
625/625 [==============================] - 132s 212ms/step - loss: 0.0188 - accuracy: 0.9944 - val_loss: 1.0524 -

In [27]:

results_lstm = lstm_model.evaluate(test_data)


782/782 [==============================] - 53s 67ms/step - loss: 0.8930 - accuracy: 0.8227


In [28]:

print('Test Acc.: {:.2f}%'.format(results_lstm[1]*100))


Test Acc.: 82.27%
